In [1]:
from wq_api import *

session = sign_in(username,password)

<Response [201]>


In [3]:
# 所有参数

# 从平台抓取的alpha数量(上限1000)
n_alpha = 200

#文件夹位置
dir_path = './refine_data'

# 数据文件位置(重点!!!,因为可能操作remain数据,请以1.csv结尾,结果结尾为3.csv,OS/IS结果结尾为4.csv)
from_path = f'{dir_path}/0_refine_1.csv'

# 回测未完成数据存储位置
remain_path = f'{dir_path}/0_remain_1.csv'

In [4]:
import os

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# 将路径转换为列表，以便修改字符
to_path_list = list(from_path)
if to_path_list[-5] == '1':
    # 修改倒数第5个字符为'3'
    to_path_list[-5] = '3'
    # 将列表转换回字符串
    to_path = ''.join(to_path_list)
else:
    raise ValueError(f"The character to be replaced is not '1'. Found '{to_path_list[-5]}' instead.")

print(f'Next available file path: {to_path}')

Next available file path: ./refine_data/0_refine_3.csv


In [5]:
alphas =  get_n_is_alphas(session, n_alpha)

In [6]:
priority_alpha_df = pd.read_csv(from_path)
priority_alpha_df = priority_alpha_df.iloc[:, 1:]
priority_alpha_df['submmitable'] = np.nan
priority_alpha_df['sharpe'] = np.nan
priority_alpha_df['fitness'] = np.nan
priority_alpha_df['id'] = np.nan

In [7]:
# 遍历 ids 并查找相应的 alphas 信息, 输出通过所有测试的alpha地址
alpha_list = []
for index, row in priority_alpha_df.iterrows():
    for alpha in alphas:
        if alpha['regular']['code'] == row['regular'] and \
            alpha['settings']['universe'] == row['universe'] and \
            alpha['settings']['decay'] == row['decay'] and \
            alpha['settings']['neutralization'] == row['neutralization']:
            priority_alpha_df.loc[index, 'submmitable'] = submittable(alpha)
            priority_alpha_df.loc[index, 'sharpe'] = alpha['is']['sharpe']
            priority_alpha_df.loc[index, 'fitness'] = alpha['is']['fitness']
            if(alpha['is']['sharpe'] > 1.2):
                if(submittable(alpha)):
                    print(alpha['id'])
                    priority_alpha_df.loc[index, 'id'] = alpha['id']
                    alpha_list.append(alpha['id'])
            break

# 保存修改后的 DataFrame
priority_alpha_df.to_csv(to_path)


In [8]:
# store remained data
remain_df = priority_alpha_df[priority_alpha_df['sharpe'].isnull()]
remain_df.to_csv(remain_path)

In [ ]:
# prod corr 记录区


In [ ]:
# 测试单因子prod_corr
# alpha_id = 'OpKN8nv'
# print(get_prod_corr_with_retries(session, alpha_id))

NameError: name 'OpKN8nv' is not defined

In [ ]:
# 测试全部可提交因子prod corr
# for alpha_id in alpha_list[0:]:
#     print(alpha_id)
#     print(get_prod_corr_with_retries(session, alpha_id))

vLaZbvb


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# 初始化 DataFrame
performance_df = pd.DataFrame(columns=['id', 'IS_Sharpe', 'OS_Sharpe', 'OS/IS_Ratio'])

In [ ]:
# 定义每批次处理的数量
batch_size = 5

# 遍历 alpha_list，按批次处理
for i in range(0, len(alpha_list), batch_size):
    batch = alpha_list[i:i + batch_size]
    
    for alpha_id in batch:
        pnl = get_pnl(session, alpha_id)

        df = pd.DataFrame(pnl.json()['records'], columns=['Date', 'PnL'])
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)

        df['Return'] = df['PnL'].pct_change()

        df['Rolling_Sharpe'] = rolling_sharpe(df['Return'])

        end_date = df.index.max()
        start_date_last_two_years = end_date - pd.DateOffset(years=2)
        start_date_is = df.index.min()
        end_date_is = start_date_last_two_years

        df_os = df[start_date_last_two_years:end_date]
        df_is = df[start_date_is:end_date_is]

        is_sharpe = df_is['Rolling_Sharpe'].mean()
        os_sharpe = df_os['Rolling_Sharpe'].mean()
        
        if os_sharpe != 0:  # 防止除零错误
            ratio_os_is = os_sharpe / is_sharpe
        else:
            ratio_os_is = float('inf')  # 或者选择其他合适的处理方式

        # 使用 pandas.concat 方法
        new_row = pd.DataFrame({
            'id': [alpha_id],
            'IS_Sharpe': [is_sharpe],
            'OS_Sharpe': [os_sharpe],
            'OS/IS_Ratio': [ratio_os_is]
        })

        performance_df = pd.concat([performance_df, new_row], ignore_index=True)
    
    # 每批次处理完后暂停 5 秒
    time.sleep(5)

Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved


In [ ]:
performance_path_list = list(from_path)
if performance_path_list[-5] == '1':
    # 修改倒数第5个字符为'4'
    performance_path_list[-5] = '4'
    # 将列表转换回字符串
    performance_path = ''.join(performance_path_list)
else:
    raise ValueError(f"The character to be replaced is not '1'. Found '{performance_path_list[-5]}' instead.")


performance_df.to_csv(performance_path)